In [1]:
import os
import pandas as pd
from requests_html import HTML

import re
from pymystem3 import Mystem
from sklearn.feature_extraction.text import CountVectorizer
from nltk.corpus import stopwords
import nltk

In [31]:
m = Mystem()
nltk.download('stopwords')
stop_words = stopwords.words('russian') + stopwords.words('english')
count_vect = CountVectorizer(stop_words=stop_words, ngram_range=(2, 2))

[nltk_data] Downloading package stopwords to /home/varsey-
[nltk_data]     fabuza/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [3]:
def lemmatize(lst):
    for n in range(len(lst)):
        lst[n] = m.lemmatize(lst[n])
        lst[n] = lst[n][:-1]
    return lst

In [4]:
def clear_text(text):
    text = re.sub(r'[^а-яА-ЯёЁa-zA-Z ]', ' ', text)
    text = text.split()
    return ' '.join(text)

In [5]:
global_counter_modify_jobs = 0

In [6]:
#функция обработки названий работ из резюме 

def hh_job_preparation(string):
    global global_counter_modify_jobs
    
    result = string.lower()

    temp_string = result

    result = result.replace("зам. ", "заместитель ")
    result = result.replace("нач. ", "начальник ")
    result = result.replace("рук. ", "руководитель ")
    result = result.replace("ген. ", "генеральный ")
    result = result.replace("гендиректор", "генеральный директор")
    result = result.replace("–", "-")
    result = result.replace(" - ", "-")

    if (result.find("(")>0 and result.find(")")>0):
        temp = ""
        not_closed = False
        for i in range(len(result)):
            if result[i] == "(":
                not_closed = True
            elif result[i] == ")":
                not_closed = False
            else:
                if not not_closed:
                    temp += result[i]
        result = temp

    if (result.find("«")>0 and result.find("»")>0):
        temp = ""
        not_closed = False
        for i in range(len(result)):
            if result[i] == "«":
                not_closed = True
            elif result[i] == "»":
                not_closed = False
            else:
                if not not_closed:
                    temp += result[i]
        result = temp

    if (result.find(",") > 0) :
        result=result.partition(',')[0]

    if (result.find(";") > 0) :
        result=result.partition(';')[0]

    if (result.find(".") > 0) :
        result=result.partition('.')[0]

    if (result.find("/") > 0) :
        result=result.partition('/')[0]

    if (result.find(" по ") > 0) :
        result=result.partition(' по ')[0]


    if (result.find("начальник ") > -1 or result.find("руководитель ") > -1):
        if (result.find("отдела") > -1):
            result = "начальник отдела"
        elif (result.find("службы") > -1):
            result = "начальник службы"
        elif (result.find("управления") > -1):
            result = "начальник управления"
        elif (result.find("подразделения") > -1):
            result = "начальник подразделения"
        elif (result.find("группы") > -1):
            result = "начальник группы"

    if (result.find("заместитель начальника") > -1 or result.find("заместитель руководителя ") > -1):
        if (result.find("отдела") > -1):
            result = "заместитель начальника отдела"
        elif (result.find("службы") > -1):
            result = "заместитель начальника службы"
        elif (result.find("управления") > -1):
            result = "заместитель начальника управления"
        elif (result.find("подразделения") > -1):
            result = "заместитель начальника подразделения"
        elif (result.find("группы") > -1):
            result = "заместитель начальника группы"

    if (result.find("developer") > -1 or result.find("programmer") > -1 ):
        result="программист"

    if (result.find("генеральный директор") > -1 or result.find("programmer") > -1 ):
        result = "генеральный директор"

    result = result.strip()

    if (result != temp_string):
        global_counter_modify_jobs += 1


    return(result)

In [7]:
#функция обработки временного периода из резюме с hh
def hh_time_preparation(string):
    # Вариант 1 - Долго работал
    # ['Март', '1986', '—', 'Май', '1989', '3', 'года', '3', 'месяца']
    # Вариант 2 - Круглые года
    # ['Февраль', '2019', '—', 'Январь', '2022', '3', 'года']
    # Вариант 3 - Круглые месяцы
    # ['Май', '2003', '—', 'Август', '2003', '4', 'месяца']
    # Вариант 4 - долго работал и по настоящее время
    # ['Июнь', '2020', '—', 'по', 'настоящее', 'время', '2', 'года', '8', 'месяцев']
    # Вариант 5 - Круглые года и по настоящее время
    # ['Июнь', '2020', '—', 'по', 'настоящее', 'время', '2', 'года']
    # Вариант 6 - Круглые месяцы и по настоящее время
    # ['Май', '2022', '—', 'по', 'настоящее', 'время', '9', 'месяцев']
    word_array = string.split()
    months = 0
    if ((word_array[4] == "настоящее") or (word_array[3] == "currently")): 
        months = 10000
    elif (len(word_array) == 9): 
        months = 12 * int(word_array[5]) + int(word_array[7])
    elif word_array[6] in ['месяца', 'месяцев', 'months', 'month']: 
        months = int(word_array[5])
    elif word_array[6] in ['года', 'лет', 'год', 'years', 'year']: 
        months = 12 * int(word_array[5])
    else:
        months = 1
        print("Некорректное резюме, проблема со временем в должности: "+ str(word_array))
    return (int(months))

In [10]:
data = []

In [11]:
def hh_resume_online_parser(input_file, client_id):
    resume = HTML(html=input_file)

    time = []
    job = []
    des = []

    resume_experience = resume.find('div[data-qa=resume-block-experience]')
    resume_experience_len = len(resume_experience)
    if (resume_experience_len == 1):
        resume_times = resume_experience[0].find('.bloko-column.bloko-column_xs-4.bloko-column_s-2.bloko-column_m-2.bloko-column_l-2')
        resume_jobs = resume_experience[0].find('div[data-qa=resume-block-experience-position]')
        resume_des = resume_experience[0].find('div[data-qa=resume-block-experience-description]')
        for i in range(len(resume_jobs)):  # смотрим места работы
            time.append(hh_time_preparation(resume_times[i].text))
            job.append(resume_jobs[i].text)
            des.append(resume_des[i].text)

    elif (resume_experience_len == 0):
        print("Не удалось открыть резюме")
        return 0
    
    else:
        print("Некорректное резюме, количество resume-block-experience = " + str(resume_experience_len))
        return 0

    # des = lemmatize(des)
    
    prof_id = 0
    for n in range(len(job)):
        data.append([client_id, prof_id, job[n], time[n], des[n]])
        prof_id += 1
    client_id += 1

In [19]:
# files = os.listdir('data\\resumes')
files = os.listdir(
    os.path.abspath(os.path.join(os.getcwd(), os.pardir)) + '/data/resumes'
)

In [20]:
files

['24d775840002c421fb0039ed1f386c33337735.html',
 '2542d4fd0001ead1ab0039ed1f515030687057.html',
 '2c776cd800028a38d80039ed1f7646436d4e77.html',
 '21947f4f0007fcd3c50039ed1f624344354d58.html',
 '3224bcf400039786160039ed1f6f7745373547.html',
 '44bd436200024053ec0039ed1f774b36584749.html',
 '6813806d0002c9bb970039ed1f557630324b74.html',
 'bb44d09a0001fa75960039ed1f736d64537632.html',
 'cf4148ce0002691bae0039ed1f72546c55594d.html',
 '7fb685eb0002c0c4ef0039ed1f5a4f6b683238.html',
 '160558610000c340dc0039ed1f5356614b3039.html',
 'aa30390800038e8f050039ed1f46685548456d.html',
 'cd255e9e000026d2020039ed1f736563726574.html',
 'd14155d900036eb5920039ed1f4e4256644674.html',
 '6b9089c10003d4b3f90039ed1f6c4d6d443632.html',
 '0ff2699d0003971f5b0039ed1f76776c637564.html',
 '0e755fa20002fca7380039ed1f36364d443043.html',
 '6aac52b10006322b710039ed1f685555773142.html',
 'a4a0c9db00035696f60039ed1f6f4c5666504d.html',
 '6bd2288c0007cc85fd0039ed1f6c6441796232.html',
 '957db86800036915190039ed1f656a387a7634

In [23]:
client_id = 0
for name in files:
    with open(os.path.abspath(os.path.join(os.getcwd(), os.pardir)) + f'/data/resumes/{name}', encoding='utf-8') as f:
    # soup = BeautifulSoup(f, 'html.parser')
        file = f.read()
        hh_resume_online_parser(file, client_id)
        client_id += 1

Некорректное резюме, проблема со временем в должности: ['Январь', '2008', '—', 'Январь', '2008', '1', 'месяц']
Некорректное резюме, проблема со временем в должности: ['Январь', '2008', '—', 'Январь', '2008', '1', 'месяц']
Некорректное резюме, проблема со временем в должности: ['Сентябрь', '2015', '—', 'Сентябрь', '2015', '1', 'месяц']
Некорректное резюме, проблема со временем в должности: ['Апрель', '2001', '—', 'Апрель', '2001', '1', 'месяц']
Некорректное резюме, проблема со временем в должности: ['Январь', '2011', '—', 'Январь', '2011', '1', 'месяц']
Некорректное резюме, проблема со временем в должности: ['Июль', '2007', '—', 'Июль', '2007', '1', 'месяц']
Некорректное резюме, проблема со временем в должности: ['Декабрь', '2019', '—', 'Декабрь', '2019', '1', 'месяц']
Некорректное резюме, проблема со временем в должности: ['Май', '2019', '—', 'Май', '2019', '1', 'месяц']
Некорректное резюме, проблема со временем в должности: ['Январь', '2015', '—', 'Январь', '2015', '1', 'месяц']
Некор

In [24]:
df = pd.DataFrame(data, columns=['client_id', 'prof_id', 'job', 'months', 'description'])

In [33]:
n_gramm = count_vect.fit_transform(df.job.to_list())
n_gramm

<4335x3759 sparse matrix of type '<class 'numpy.int64'>'
	with 7123 stored elements in Compressed Sparse Row format>

In [34]:
df

,client_id,prof_id,job,months,description
0,0,0,бригадир комплексной бригады,10000,"Выращивание с/х культур:оз.пшеница,оз.ячмень,я..."
1,1,0,Заместитель генерального директора по общим во...,9,Решение вопросов касающихся производственно - ...
2,1,1,Инженер - механик,8,Перерегистрация автотракторной техники. Списан...
3,1,2,Заведующий машинным двором,26,"Ремонт автомобилей, тракторов и с/х техники, с..."
4,1,3,Заведующий прудовым хозяйством,7,Руководство коллективом прудового хозяйства. В...
...,...,...,...,...,...
4330,999,6,Помощник руководителя (начальник службы охраны),27,- организация работы по охране и физической за...
4331,999,7,Менеджер по продажам услуг,9,"- ведение документооборота, проверка отчетов у..."
4332,999,8,Главный механик,73,"- организация, управление автопарком, обеспече..."
4333,999,9,о/у ОБЭП. инспектор обслуживания видеонабл. сп...,155,- выявление и пресечение преступлений экономич...


In [155]:
df.to_csv('data.csv')